# Fields as Curried Functions

In a 1994 paper, Gary T Leavens explained, in a wonderfully graspable manner, that Fields in physics can actually be viewed as curried functions. If you understand what curried functions are, then you already have a good idea of what fields are—these sort of bridges are incredibly helpful to those of us peering in from both sides of the divide (for me, from the CS side). The examples were in scheme and so the syntax inversion might make it a bit difficult for the unpracticed (such as myself) to follow, hence my translating (and hopefully, helpful to you too). My examples here are in F#, where units of measure are especially helpful in making things clearer.

##Table of Contents
* [Fields as Curried Functions](#Fields-as-Curried-Functions)
* [Gravity](#Gravity)
	* [Graphing the near Earth Gravitational Pull](#Graphing-the-near-Earth-Gravitational-Pull)
	* [Not Falling is Unnatural](#Not-Falling-is-Unnatural)
* [Vector Fields](#Vector-Fields)
	* [Finally, On Currying vs Fields](#Finally,-On-Currying-vs-Fields)


#Gravity

First some definitions:

In [126]:
[<Measure>] type kg
[<Measure>] type m
[<Measure>] type s
[<Measure>] type N = kg * m / s^2

let G = 6.674e-11<N * m^2 / kg^2>


val G ∈ float<N m²/kg²> = 6.674e-11


In [129]:
let grav_force (m1:float< kg>) (r:float<m>) (m2:float<kg>) =        
    if r = 0.<m> then 0.<N> 
    else ((m1 * m2) * G) / (r * r)


val grav_force ∈ m1∈float<kg> ⟹ r∈float<m> ⟹ m2∈float<kg> ⟹ float<N>


This is a curried function since it's a function that takes a value and returns a function specialized to the passed in value.

In [127]:
let asideHtm s = {Html = sprintf "<div style='border-radius:0.25em;background-color:#FFD954;color:#7F3300;border: 1px solid;font-size:110%%;padding:0.2em 0.6em 0.3em'>%s</div>" s}


val asideHtm ∈ s∈string ⟹ HtmlOutput


In [178]:
//
asideHtm "<b>Briefly</b>: a curried function such as 
plus: (+) ∈ (int ⟹ int ⟹ int), allows us to say, pre-apply 5 to it. As in, e.g., f = (+) 5. 
Then we can do f 3 = 8 or f 10 = 15 etc. Basically, with the preapplication of 5 to (+), we get back a 
new function: int ⟹ int which takes a number and returns 5 + that number. Easy yes?"

Briefly : a curried function such as 
plus: (+) ∈ (int ⟹ int ⟹ int), allows us to say, pre-apply 5 to it. As in, e.g., f = (+) 5. 
Then we can do f 3 = 8 or f 10 = 15 etc. Basically, with the preapplication of 5 to (+), we get back a 
new function: int ⟹ int which takes a number and returns 5 + that number. Easy yes?

Returning to our function `grav_force`, when we put it into F# we got the type: 

`val grav_force ∈ m1∈float<kg> ⟹ r∈float<m> ⟹ m2∈float<kg> ⟹ float<N>`

So everything looks good, it takes a mass (such as Earth) and returns a function that takes a distance (e.g. at the surface) which itself returns a function: one from masses (your mass) to Forces (your weight). This function is in fact the definition of a (_scalar_) gravitational field but more on that later. We can _specialize_ the function and get earth's field.

In [130]:
let earth_mass = 5.96e24<kg>
let earth_rad = 6.37e6<m>
let earth_grav = grav_force earth_mass


val earth_mass ∈ float<kg> = 5.96e+24
val earth_rad ∈ float<m> = 6370000.0
val earth_grav ∈ (float<m> ⟹ float<kg> ⟹ float<N>)


Similarly, we can specialize to the surface (by applying a distance to `earth_grav`) or we can compute the mass of an object at a particular distance by passing in a distance _and_ a mass. For example, noticing that `N = kg m/s²`, we can compute Earth's accelaration at surface simply by passing in _1kg_ and the Earth's radius for a value of 9.802877992 N (which is just m/s² for this case). But we can do more interesting things with curried functions. We can flip the field equation to compute the gravitational force at a list of distances from earth's surface (far much more flexible than how physics is typically taught isn't it):

In [220]:
let flip f a b = f b a
let flipped_field = flip earth_grav 1.<kg>


val flip ∈ f∈('a ⟹ 'b ⟹ 'c) ⟹ a∈'b ⟹ b∈'a ⟹ 'c
val flipped_field ∈ (float<m> ⟹ float<N>)


In [181]:
[for dist in 0.0<m>..1e5<m>..1e6<m> -> 6.37e6<m> + dist] |> List.map flipped_field


val it ∈ float<N> list =
  [9.802877992; 9.502194172; 9.215135446; 8.940890874; 8.678708962; 8.42789251;
   8.187793968; 7.957811259; 7.737383994; 7.525990059; 7.323142521]

The map applies our function to each distance in the list to get a list of forces. So at a distance of 1000km, the gravitational acceleration is ~7.32 m/sec². 

In [182]:
//
asideHtm "<b>Trivia</b>: Mount Everest's height is 8848 meters, g = 9.776 m/sec² there—only 0.28% 
weaker than on the surface."

Trivia : Mount Everest's height is 8848 meters, g = 9.776 m/sec² there—only 0.28% 
weaker than on the surface.

Below you can look at the gravitational acceleration on the surface of the moon and Mars. The moon has a mere acceleration of 1.62 m/s² and Mars is only just over 2x that—it is for this reason that many are concerned over the health effects (on skeletomuscular integrity) of an extended mission to Mars. 

Additionally, we can look at the Earth/Moon system. The Moon and the Earth both exert a force of $2 \times10^{20}\,N$ on each other. Both are forever falling towards each other, with the condition that—on the odd chance that—if ever they should meet, things would not end well for both of them. A tragic love story if ever there was one.

In [221]:
let mass_moon = 7.3459e22<kg>
let mass_mars = 6.41693e23<kg>
let rad_moon = 1737.5<m> * 1000.
let radius_mars = 3386.<m> * 1000.

let mean_moon_dist = 3.85e8<m>

grav_force mass_moon rad_moon    1.<kg>  , 
grav_force mass_mars radius_mars 1.<kg>  ,
grav_force earth_mass mean_moon_dist mass_moon


val mass_moon ∈ float<kg> = 7.3459e+22
val mass_mars ∈ float<kg> = 6.41693e+23
val rad_moon ∈ float<m> = 1737500.0
val radius_mars ∈ float<m> = 3386000.0
val mean_moon_dist ∈ float<m> = 385000000.0
val it ∈ float<N> ⨯ float<N> ⨯ float<N> =
  (1.623983408, 3.735421349, 1.971314948e+20)

## Graphing the near Earth Gravitational Pull

To make things a bit more concrete than a list of numbers, I graph some commonly known locations and the gravitational acceleration felt there. Looking at these, I can't help but think of the common conception (held by myself for a long time too) that space is "out there" and far away; when really, space (LEO, at least) is literally within walking distance. Most satellites, the International Space Station included, aren't floating in space; they're still deep within Earth's gravitational well!

In [184]:
let pois = [|"~Low Earth Orbit Min",   160.<m>  
             "~Low Earth Orbit Max",  2000.<m>  
             "GEO Synch"           , 40000.<m>
             "GPS"                 , 20350.<m>
             "ISS Avg"             ,  382.5<m>
             "Hubble Space Telescope",595.<m>|] 

pois |> Array.map (snd >> (*) 1000. >> (+) earth_rad >> flipped_field) 


val pois ∈ (string ⨯ float<m>) [] =
  [|("~Low Earth Orbit Min", 160.0); ("~Low Earth Orbit Max", 2000.0);
    ("GEO Synch", 40000.0); ("GPS", 20350.0); ("ISS Avg", 382.5);
    ("Hubble Space Telescope", 595.0)|]
val it ∈ float<N> [] =
  [|9.328377215; 5.67781903; 0.1849942672; 0.557133861; 8.723751306;
    8.199553815|]

In [185]:
//a script, mix of JS and F# to generate a graph

open System

let inline roundn (n:int) x = Math.Round(float x, n)

let adjp (s:string) = if s.[..2] = "GEO" then 30 else 0

let poi_annotation adj d g s = 
    let arrow = """ 
          ctx.beginPath();
          ctx.moveTo(o.left, o.top);
          ctx.lineTo(o.left, o.top - 10);
          ctx.lineTo(o.left + 10, o.top - 5);
          ctx.lineTo(o.left, o.top);
          ctx.fillStyle = "#000";
          ctx.fill();""" 
    sprintf
     """$("#placeholder").append("<div style='position:absolute;left:" + (o.left + 4) + "px;top:" + (o.top - %d) + "px;color:#666;font-size:smaller'>%s (%Akm, %Am/s²)</div>");
          %s""" adj s d g arrow;

let accels = [for dist in 0.0<m>..1e5<m>..5e7<m> do
                let d = (6.37e6<m> + dist) 
                let g = (earth_grav d 1.<kg>) 
                yield sprintf "[%A,%A]" (roundn 3 (d/1000. - 6370.<m>)) (roundn 3 g)] 

let accels_str = String.Join(",", accels) 

let annots = 
  [for (l,d) in pois -> 
    let g = flipped_field (earth_rad + d * 1000.)
    sprintf "o = plot.pointOffset({ x: %A, y: %A});\n%s;"
             d g (poi_annotation (adjp l) (roundn 2 d) (roundn 2 g) l)]
             
let annots_str = "var ctx = plot.getCanvas().getContext(\"2d\");\nvar o;\n" + String.Join("\n\n", annots)

let flotTemplate = """

<div id="placeholder" style="width:800px;height:620px;"></div>
<div style='text-align:center'>km</div>
<script type="text/javascript"> 

require(['https://cdnjs.cloudflare.com/ajax/libs/flot/0.8.3/jquery.flot.js'], function(flot) {

  plot = $.plot("#placeholder", 
   [{ data:  [__ACCEL__], label: "Gravitational Accel = 0.0m" }], 
    { series:{lines:{show: true}},
	  grid:  {hoverable: true},
      yaxis: { tickFormatter: function(val, axis) { return val < axis.max ? val.toFixed(2) : "m/s²";}}
    });

  var legends = $("#placeholder .legendLabel");

  legends.each(function () {
   // fix the widths so they don't jump around
   $(this).css('width', $(this).width());
  });

  var updateLegendTimeout = null;
  var latestPosition = null;

  function updateLegend() {

   updateLegendTimeout = null;

   var pos = latestPosition;

   var axes = plot.getAxes();
   if (pos.x < axes.xaxis.min || pos.x > axes.xaxis.max ||
    pos.y < axes.yaxis.min || pos.y > axes.yaxis.max) {
    return;
   }

   var i, j, dataset = plot.getData();
   
   for (i = 0; i < dataset.length; ++i) {
    var series = dataset[i];

    // Find the nearest points, x-wise
    for (j = 0; j < series.data.length; ++j) {
     if (series.data[j][0] > pos.x) {break;}
    }

    // Now Interpolate
    var y,p1 = series.data[j - 1], p2 = series.data[j];

    if (p1 == null) {
     y = p2[1];
    } else if (p2 == null) {
     y = p1[1];
    } else {
     y = p1[1] + (p2[1] - p1[1]) * (pos.x - p1[0]) / (p2[0] - p1[0]);
    }

    legends.eq(i).text(series.label.replace(/=.*/, "= " + y.toFixed(2) + "m/s² (" + pos.x.toFixed(1) +"km)"));
   }
  }
  
  __ANNOTS__

  $("#placeholder").bind("plothover",  function (event, pos, item) {
   latestPosition = pos;
   if (!updateLegendTimeout) {
    updateLegendTimeout = setTimeout(updateLegend, 50);
   }
  });
});
</script>
"""

{Html = flotTemplate.Replace("__ACCEL__",accels_str).Replace("__ANNOTS__", annots_str)}

km

## Not Falling is Unnatural

> There is an art, it says, or rather, a knack to flying. The knack lies in learning how to throw yourself at the ground and miss. Pick a nice day, [The Hitchhiker's Guide to the Galaxy] suggests, and try it.

>The first part is easy. All it requires is simply the ability to throw yourself forward with all your weight, and the willingness not to mind that it's going to hurt.

>That is, it's going to hurt if you fail to miss the ground. Most people fail to miss the ground, and if they are really trying properly, the likelihood is that they will fail to miss it fairly hard.

>Clearly, it is the second part, the missing, which presents the difficulties..."

>--Douglas Adams

In [186]:
//
asideHtm "How can those on the ISS feel ~89% of what we get here on the surface at the
same time as experiencing weightlessness? If you're like me, when you asked, you gotten the 
standard \"because they're in free-fall\". I find that explanation lacking; 
it fails to elaborate on what about freefall 
makes one weightless. The <a href='http://physics.stackexchange.com/questions/29929/gravity-on-the-international-space-station'>
clearest analogy</a> I've ever run into links freefall to the above quoted passage 
from THHGTH. Things in orbit around each other are one of the few things able to fall and 
manage to miss the ground. The key to appreciating free-fall is in realizing that things
in orbit and every day falling (ignoring friction and such) are the same thing. 
Unlike how THHGTH recommends acheiving flight however,for things in orbit, it is not so much 
that they miss the ground as it's that the ground keeps moving out of 
the away (it helps to look at it that way instead of as the symmetric situation of 
horizontal vs vertical velocity).
<br/><br/>
To fully understand the phenomenon however, we need disentangle the two meanings of weight 
typically mixed together. There
is weight as in the force exerted by participating in a gravitational field 
and there is the everyday notion of weight
we feel. They're related since our weight comes from being party to a gravitational field 
while something else pushes
back just as hard as the Earth is pulling.
The problem arises when we combine our notion of fall (downwards) with the idea that weight 
has to feel like
something. But in fact, just being in a (uniform) gravitational field is 
not enough to \"feel solid\", you need
a counter opposing force resisting your inertia. In the everyday world,
this is provided by the Normal Force of say, a chair or the ground
pushing back at you, resisting gravity and keeping you from falling. 

Things that are falling (this includes an apple, a jumper or the moon or the Earth)
have no <i>'felt'</i> weight, the reason the ISS and other objects in orbit manage to achieve 
weightlessness is because the
ground manages to move away before they get a chance to hit it. This is something
we could all manage without having
to move at incredible horizontal velocities if we somehow figured out how to jump
and miss the ground by accident."

How can those on the ISS feel ~89% of what we get here on the surface at the
same time as experiencing weightlessness? If you're like me, when you asked, you gotten the 
standard "because they're in free-fall". I find that explanation lacking; 
it fails to elaborate on what about freefall 
makes one weightless. The 
clearest analogy I've ever run into links freefall to the above quoted passage 
from THHGTH. Things in orbit around each other are one of the few things able to fall and 
manage to miss the ground. The key to appreciating free-fall is in realizing that things
in orbit and every day falling (ignoring friction and such) are the same thing. 
Unlike how THHGTH recommends acheiving flight however,for things in orbit, it is not so much 
that they miss the ground as it's that the ground keeps moving out of 
the away (it helps to look at it that way instead of as the symmetric situation of 
horizontal vs vertical velocity).
 
To fully understand the phenomenon however, we need disentangle the two meanings of weight 
typically mixed together. There
is weight as in the force exerted by participating in a gravitational field 
and there is the everyday notion of weight
we feel. They're related since our weight comes from being party to a gravitational field 
while something else pushes
back just as hard as the Earth is pulling.
The problem arises when we combine our notion of fall (downwards) with the idea that weight 
has to feel like
something. But in fact, just being in a (uniform) gravitational field is 
not enough to "feel solid", you need
a counter opposing force resisting your inertia. In the everyday world,
this is provided by the Normal Force of say, a chair or the ground
pushing back at you, resisting gravity and keeping you from falling. 

Things that are falling (this includes an apple, a jumper or the moon or the Earth)
have no 'felt' weight, the reason the ISS and other objects in orbit manage to achieve 
weightlessness is because the
ground manages to move away before they get a chance to hit it. This is something
we could all manage without having
to move at incredible horizontal velocities if we somehow figured out how to jump
and miss the ground by accident.

# Vector Fields

At this point, I can't help but note that I've veered a bit far afield. The original intention of this piece was to connect curried functions from functional programs to fields in physics and yet, here I am talking about how falling is flying with no pesky forces in the way. Nonetheless, I hope the above has been an effective demonstration of the advantage of a computational approach to learning topics commonly thought of as challenging (there'll be more such demonstrations in the below). In reality, much of the difficulty is incidental instead of necessary: from the student, it is grappling with inconsistent and often unmotivated notation, as well as plain general unfarmliarity and from the teacher: the baggage of being wedded to centuries old tradition of how subjects must be taught. Much pointless complexity arises from the interaction of all those variables.

Right, on topic. The real world has more than just one dimension. Depending on whom you ask, it can be anywhere from 3 or 4 to 11 to 26 or more or less. Most real world (classical) problems settle on 3 however. And here, again we see the advantage of a computational approach. It takes only a few lines to generalize our methodology to vectors (and though it's general to N-dimensions, only for 3—or 2 in a few places—does it really makes sense for the operations we're performing).

In [132]:
//util
[<FunScript.JS>]
let inline cube r = r * r * r

[<FunScript.JS>]
let inline squared r = r * r

[<FunScript.JS>]
let inline vec_mag (v:list<float<_>>) = List.foldBack ((float >> squared >> (+))) v 0. |> sqrt


val inline cube ∈
  r∈ ^a ⟹  ^c
    when  ^a ∈ (static member ( ⨯ ) ∈  ^a ⨯  ^a ⟹  ^b) and
         ( ^b or  ^a) ∈ (static member ( ⨯ ) ∈  ^b ⨯  ^a ⟹  ^c)
val inline squared ∈
  r∈ ^a ⟹  ^b when  ^a ∈ (static member ( ⨯ ) ∈  ^a ⨯  ^a ⟹  ^b)
val inline vec_mag ∈ v∈float<'u> list ⟹ float


In [224]:
let grav_field (m1:float<kg>) (r:float<m> list) (m2:float<kg>) =        
    let mag = vec_mag r * 1.<m>
    let scale = (-G * m1 * m2) / cube mag
    r |> List.map ((*) scale)


val grav_field ∈
  m1:float<kg> ⟹ r:float<m> list ⟹ m2:float<kg> ⟹ float<N> list

The function is the same as before except that, instead of returning a single number, we now return a list of numbers, representing our force vector. The input for distance has also been replaced with a vector. Below, we apply random masses at different locations to get force vectors. Everything is working correctly.

In [189]:
grav_field earth_mass [1.<m>; 0.<m>; earth_rad] 68.0<kg> ,
grav_field earth_mass [1.<m>; 12.<m>; earth_rad] 1.0<kg> 



val it ∈ float<N> list ⨯ float<N> list =
  ([-0.0001046461073; 0.0; -666.5957035],
   [-1.538913343e-06; -1.846696011e-05; -9.802877992])

In [190]:
let radii = [|0.0<m>..1000.0<m>..100000.0<m>|]
let masses = [|10.<kg>..1000.0<kg>..100000.<kg>|]

let xss = [for _ in 1..masses.Length -> Array.copy radii]
let yss = [for m in 0..masses.Length-1 -> Array.create radii.Length masses.[m]]
  
let zss = [|for x in 0..masses.Length - 1 -> 
            [|for y in 0..radii.Length - 1 -> 
              earth_grav (xss.[x].[y] * 1000. + earth_rad) yss.[x].[y]|]|]


val radii ∈ float<m> [] =
  [|0.0; 1000.0; 2000.0; 3000.0; 4000.0; 5000.0; 6000.0; 7000.0; 8000.0;
    9000.0; 10000.0; 11000.0; 12000.0; 13000.0; 14000.0; 15000.0; 16000.0;
    17000.0; 18000.0; 19000.0; 20000.0; 21000.0; 22000.0; 23000.0; 24000.0;
    25000.0; 26000.0; 27000.0; 28000.0; 29000.0; 30000.0; 31000.0; 32000.0;
    33000.0; 34000.0; 35000.0; 36000.0; 37000.0; 38000.0; 39000.0; 40000.0;
    41000.0; 42000.0; 43000.0; 44000.0; 45000.0; 46000.0; 47000.0; 48000.0;
    49000.0; 50000.0; 51000.0; 52000.0; 53000.0; 54000.0; 55000.0; 56000.0;
    57000.0; 58000.0; 59000.0; 60000.0; 61000.0; 62000.0; 63000.0; 64000.0;
    65000.0; 66000.0; 67000.0; 68000.0; 69000.0; 70000.0; 71000.0; 72000.0;
    73000.0; 74000.0; 75000.0; 76000.0; 77000.0; 78000.0; 79000.0; 80000.0;
    81000.0; 82000.0; 83000.0; 84000.0; 85000.0; 86000.0; 87000.0; 88000.0;
    89000.0; 90000.0; 91000.0; 92000.0; 93000.0; 94000.0; 95000.0; 96000.0;
    97000.0; 98000.0; 99000.0; ...|]
val masses ∈ float<kg> [] =


In [191]:
let jupiter_grav = grav_force 1.89813e27<kg>
let jupiter_rad = 6.9173e7<m>
  
let zss2 = [|for x in 0..masses.Length - 1 -> 
             [|for y in 0..radii.Length - 1 -> 
               jupiter_grav (xss.[x].[y] * 1000. + jupiter_rad) yss.[x].[y]|]|]
               
let zss3 = [|for x in 0..masses.Length - 1 -> 
             [|for y in 0..radii.Length - 1 -> 
               grav_force mass_moon (xss.[x].[y] * 1000. + rad_moon) yss.[x].[y]|]|]


val jupiter_grav ∈ (float<m> ⟹ float<kg> ⟹ float<N>)
val jupiter_rad ∈ float<m> = 69173000.0
val zss2 ∈ float<N> [] [] =
  [|[|264.7518071; 257.2598833; 250.0815264; 243.1994788; 236.5976538;
      230.2610421; 224.1756258; 218.328301; 212.7068074; 207.2996638;
      202.0961093; 197.0860502; 192.2600105; 187.6090872; 183.1249093;
      178.7996002; 174.6257429; 170.5963482; 166.7048255; 162.9449557;
      159.3108667; 155.7970099; 152.3981395; 149.1092926; 145.9257712;
      142.8431255; 139.8571382; 136.96381; 134.1593465; 131.4401453;
      128.8027851; 126.2440141; 123.7607406; 121.3500236; 119.0090635;
      116.735195; 114.5258784; 112.3786933; 110.2913316; 108.2615915;
      106.2873713; 104.3666647; 102.4975548; 100.67821; 98.90687921;
      97.18188759; 95.5016328; 93.86458115; 92.26926409; 90.71427496;
      89.19826587; 87.71994476; 86.27807267; 84.87146115; 83.49896978;
      82.15950388; 80.85201233; 79.57548548; 78.32895324; 77.1114832;
      75.92217891; 74.76017821; 73

## Finally, On Currying vs Fields

In [192]:
let tojsliststr (xs:'a seq) = "[" + String.Join(",",xs) + "]"
let toliststr (xs:'a seq) = xs |> Seq.map tojsliststr |> tojsliststr

let xstr = toliststr xss  
let ystr = toliststr yss  
let zstr = toliststr zss
let zstr2 = toliststr zss2
let zstr3 = toliststr zss3


val tojsliststr ∈ xs∈seq<'a> ⟹ string
val toliststr ∈ xs∈seq<#seq<'a0>> ⟹ string
val xstr ∈ string =
  "[[0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,"+[59540 chars]
val ystr ∈ string =
  "[[10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10"+[59528 chars]
val zstr ∈ string =
  "[[98.028779921581,73.2314252102016,56.7781902995993,45.305745"+[170807 chars]
val zstr2 ∈ string =
  "[[264.751807099448,257.259883284856,250.08152639014,243.19947"+[170619 chars]
val zstr3 ∈ string =
  "[[16.2398340789814,6.54218707366402,3.50969043120323,2.184403"+[170990 chars]

In [207]:
let cols = "{fill_colors:['blue', 'green', 'red','orange','yellow']}"
{Html = String.Format("
<link rel='stylesheet' href='https://rawgit.com/domitry/elegans/master/examples/common.css'>
<div id='visz'>
</div>
<script type='text/javascript'>
require(['http://cdnjs.cloudflare.com/ajax/libs/three.js/r66/three.min.js',
         'https://raw.githubusercontent.com/domitry/elegans/master/release/elegans.min.js',
         'https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.6/d3.js'], function(THREE, Elegans,d3){{         
    var data = {{ }};
    var data2 = {{}};
    var data3 = {{}}
    data.x = {0}
    data.y = {1}
    data.z = {2}
    
    data2.x = {0}
    data2.y = {1}
    data2.z = {3}
    
    data3.x = {0}
    data3.y = {1}
    data3.z = {5}
    
    var stage = new Elegans.Stage(d3.select('#visz')[0][0]);
var sf1 = new Elegans.Surface(data, {4});
var sf2 = new Elegans.Surface(data2,{4});
var sf3 = new Elegans.Surface(data3,{4});
stage.add(sf1);
stage.add(sf2);
stage.add(sf3);
stage.render();

        }}); 
         </script>",xstr, ystr, zstr, zstr2, cols,zstr3)}

In [1]:
#r @"C:\Users\sir.deenicus\Documents\Visual Studio 2013\open\funscript\FunScript.dll"
#r @"C:\Users\sir.deenicus\Documents\Visual Studio 2013\open\funscript\FunScript.Interop.dll"
#r @"C:\Users\sir.deenicus\Documents\Visual Studio 2013\open\funscript\packages\FunScript.TypeScript.Binding.lib\lib\net40\FunScript.TypeScript.Binding.lib.dll"
open FunScript
open FunScript.TypeScript

In [123]:
[<FunScript.JS>]
let main () =   
    let canvas = Globals.document.getElementById("canv") :?> HTMLCanvasElement 

    let ctx = canvas.getContext_2d()      

    let drawArrowHead(x1,y1,x2,y2) =     
      
      let rad = (atan2 (y2-y1) (x2-x1)) + (if x2>=x1 then 1. else -1.)*System.Math.PI/2.;
       
      ctx.save();
      ctx.beginPath();
      ctx.translate(x2,y2);
      ctx.rotate(rad);
      ctx.moveTo(0.,0.);
      ctx.lineTo(5.,8.);
      ctx.lineTo(-5.,8.);
      ctx.closePath();
      ctx.restore();
      ctx.fill();
      ()

    let drawLine(x1,y1,x2,y2,col:string) =
      ctx.strokeStyle <- col;
      ctx.fillStyle <- col;
      ctx.lineWidth <- 1.;

      // draw the line
      ctx.beginPath();
      ctx.moveTo(x1,y1);
      ctx.lineTo(x2,y2);
      ctx.stroke();
      (x1,y1,x2,y2)

    let getMousePos(e:MouseEvent) =
        let rect = canvas.getBoundingClientRect();
        e.clientX - rect.left, e.clientY - rect.top

    let drawCircle(x,y) =
        ctx.beginPath()
        ctx.arc(x,y,5.,0.,2. * System.Math.PI);
        ctx.fill()
        ctx.stroke()
        ()

    let infdiv = Globals.document.getElementById("infdiv")

    canvas.addEventListener_mousemove(fun e ->  
        let x,y = getMousePos e 
        infdiv.innerHTML <- x.ToString() + ", " + y.ToString()
        null)  
    
    let drawArrow = drawLine >> drawArrowHead
    
    canvas.addEventListener_mousedown(fun e ->   
        drawCircle(getMousePos e); 
        let x,y = getMousePos e
        drawArrow(0.,0.,x,y,"rgba(0,0,255,0.3)")
        drawArrow(x,y, x + 50.,y+20.,"red")
        null)      
    ()
      
let js = Compiler.compileWithoutReturn (<@main()@>)  //(<@ MyNamespace.MyModule.SomeFunc @>)

let html = """
<div id="infdiv" style="width:100px;height:100px"/>
<canvas id="canv"" width="300" height="300" style="border-width:1px;border-style:solid"/>

"""




val main ∈ unit ⟹ unit
val js ∈ string =
  "var TupleDouble_Double_Double_Double_String, TupleDouble_Doub"+[8176 chars]
val html ∈ string =
  "
<div id="infdiv" style="width:100px;height:100px"/>
<canvas "+[83 chars]

In [103]:
html + "<script>" + js + "</script>"


val it ∈ string =
  "
<div id="infdiv" style="width:100px;height:100px"/>
<canvas id="canv"" width="300" height="300" style="border-width:1px;border-style:solid"/>
<script>var TupleDouble_Double_Double_Double, TupleDouble_Double, MathJS__Atan2$, LanguagePrimitives__UnboxGeneric$HTMLCanvasElement_HTMLCanvasElement_, FSI_0151__main$;
  FSI_0151__main$ = (function (unitVar0)
  {
    var canvas = LanguagePrimitives__UnboxGeneric$HTMLCanvasElement_HTMLCanvasElement_(((window.document).getElementById("canv")));
    var ctx = (canvas.getContext("2d"));
    var drawArrowHead = (function (tupledArg)
    {
      var x1 = tupledArg.Items[0.000000];
      var y1 = tupledArg.Items[1.000000];
      var x2 = tupledArg.Items[2.000000];
      var y2 = tupledArg.Items[3.000000];
      var _17;
      var _27;
      var _28;
      var _29;
      if ((x2 > x1)) 
      {
        _29 = 1.000000;
      }
      else
      {
        _29 = -1.000000;
      };
      _28 = (_29 ⨯ 3.141593);
      _27 = (_28 / 2.0

In [124]:
{Html = html + "<script>" + js + "</script>"}

In [135]:
[<FunScript.JS>]
let static_field constant (m1:float<'v>) (r:float<m> list) (m2:float<'v>) =        
    let mag = vec_mag r * 1.<m>  
    let scale = (constant * m1 * m2) / cube mag
    r |> List.map (( * ) scale)


val static_field ∈
  constant:float<'u> ⟹
    m1:float<'v> ⟹
      r:float<m> list ⟹ m2:float<'v> ⟹ float<'u 'v²/m²> list

In [136]:
let gr (m:float<kg>) (r:list<float<m>>) (m2:float<kg>) = static_field -G m r m2
gr earth_mass [1.<m>; 0.<m>; earth_rad] 68.<kg>


val gr ∈ m∈float<kg> ⟹ r∈float<m> list ⟹ m2∈float<kg> ⟹ float<N> list
val it ∈ float<N> list = [-0.0001046461073; 0.0; -666.5957035]


In [ ]:
[<Measure>] type C

let k = 9e9<N * m ^2 / C^2> 

let cr (q:float<C>) (r:list<float<m>>) (q2:float<C>) = static_field k q r q2                                                                                   
cr 1.<C> [1.<m>] 1.<C> = [9e9<N>]

let inline to_cm d = float d * 0.01<m>
let inline uC c = 1e-6<C> * float c
let inline nanoC c = 1e-9<C> * float c

cr (nanoC -6.25<C>) [to_cm 61.7<m>] (nanoC -6.25<C>)

let inline sub a b = List.map2 (-) a b

cr (uC 3) (sub [to_cm 2.5<m>;0.<m>] [to_cm 0.<m>;to_cm 2.5<m>]) (uC 3)
uC 3 * uC 3 *  k 

let electric_field_at_q (r,q) (r', q') = cr q (sub r r') q'  

let electric_field_at_r r (r', q') = cr (uC 1) (sub r r') q'  
  
let qs = [[0.;2.5], -3.; [0.; 0.],3.; [2.5;0.],-3.] |> List.map (fun (v,q) -> List.map to_cm v, uC q)

let sum_forcevecs = List.fold (List.map2 (+)) (List.init 2 (konst 0.<N>)) 
let compq rq qs = qs |> List.map (electric_field_at_q rq) |> sum_forcevecs 
let compr qs r = qs |> List.map (electric_field_at_r r) |> sum_forcevecs

let qf = qs |> compq ([to_cm 2.5<m>; to_cm 2.5<m>], uC 3) 



In [195]:
 let xdat = xss |> Array.concat |> tojsliststr
 let ydat = yss |> Array.concat |> tojsliststr
 let zdat = zss |> Array.concat |> tojsliststr



val xdat ∈ string =
  "[0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,11000,1"+[59340 chars]
val ydat ∈ string =
  "[10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,"+[59328 chars]
val zdat ∈ string =
  "[98.028779921581,73.2314252102016,56.7781902995993,45.3057454"+[170607 chars]

In [196]:
{Html = sprintf "
<link rel='stylesheet' href='https://rawgit.com/domitry/elegans/master/examples/common.css'>
<div id='vis2'>
</div>
<script type='text/javascript'>
require(['http://cdnjs.cloudflare.com/ajax/libs/three.js/r66/three.min.js',
         'https://raw.githubusercontent.com/domitry/elegans/master/release/elegans.min.js',
         'https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.6/d3.js'], function(THREE, Elegans,d3){         
    var data = {}; 
    var data = {x:%s, y:%s, z:%s};
    d3.select('#vis2').datum(data).call(Elegans.LinePlot.thickness(2)) 
        }); 
         </script>" xdat ydat zdat}

In [197]:

earth_grav (1000.<m> * 1000. + earth_rad) (100.<kg>),
earth_grav (0.<m> * 1000. + earth_rad) (100.<kg>)


val it ∈ float<N> ⨯ float<N> = (732.3142521, 980.2877992)


In [198]:
{Html = "
<link rel='stylesheet' href='https://rawgit.com/domitry/elegans/master/examples/common.css'>
<div width='400px' height='300px' id='vis2'>
</div>
<script type='text/javascript'>
 
	var WIDTH = 400,
	    HEIGHT = 300;

	// set some camera attributes
	var VIEW_ANGLE = 45,
	    ASPECT = WIDTH / HEIGHT,
	    NEAR = 0.1,
	    FAR = 10000;

	// get the DOM element to attach to
	// - assume we've got jQuery to hand
	var $container = $('#vis2');
    var renderer = new THREE.WebGLRenderer();
    renderer.setSize(WIDTH, HEIGHT);
    $container.append(renderer.domElement);
    var camera = new THREE.PerspectiveCamera(45, WIDTH / HEIGHT, 1, 500);
    camera.position.set(0, 0, 100);
    camera.lookAt(new THREE.Vector3(0, 0, 0));
    var scene = new THREE.Scene();
    
    var material = new THREE.LineBasicMaterial({
        color: 0x00ffff
    });
    
    
     var material2 = new THREE.LineBasicMaterial({
        color: 0xffffff
    });
    
    var geometry = new THREE.Geometry();
    geometry.vertices.push(new THREE.Vector3(-10, 0, 0));
    geometry.vertices.push(new THREE.Vector3(0, 10, 0));
    geometry.vertices.push(new THREE.Vector3(10, 0, 0));
    
        var line = new THREE.Line(geometry, material);
        scene.add(line);
        
        
        var geometry2 = new THREE.Geometry();
        geometry2.vertices = [];
        geometry2.vertices.push(new THREE.Vector3(-10, 0, 30));
    geometry2.vertices.push(new THREE.Vector3(0, 30, 0));
    
    
        var line2 = new THREE.Line(geometry2, material2);
         scene.add(line2);
       
    renderer.render(scene, camera); 
         </script>"}

In [199]:
open System

let txt = IO.File.ReadAllText("Untitled1.html")


val txt ∈ string =
  "<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />
<titl"+[213384 chars]

In [200]:
let ntxt = txt.Replace("&#39;", "'").Replace("&quot;","\"").Replace("&lt;", "<").Replace("&gt;", ">").Replace("&apos;","'")
IO.File.WriteAllText("Untitled1.htm",ntxt)


val ntxt ∈ string =
  "<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />
<titl"+[212459 chars]
val it ∈ unit = ()
